# Project 2
### Antonia Sunseri 

In [10]:
# Loading Packages 
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio 
import dash
from dash import Dash, dcc, html, Input, Output
pio.templates.default = "plotly_white"

In [11]:
# Reading in dataframe
df = pd.read_csv(r'C:\Users\anton\OneDrive - University of Denver\Spring Quarter 2025\COMP 4433\Antonia_Sunseri_Project 2\country_vaccinations.csv')

# First 6 rows
df.head()


latest_df = df.sort_values('date').groupby('country').tail(1)

df_cleaned = df.dropna()

In [12]:
# Save to new dataframe
df_cleaned.to_csv("cleaned_country_vaccinations.csv", index=False)

In [13]:
app = dash.Dash(__name__)
app.title = "COVID-19 Global Vaccination Dashboard"

In [14]:
# App Layout
app.layout = html.Div([
    # App title
    html.H1("Global COVID-19 Vaccination Dashboard", style={'textAlign': 'center'}),

    # Control panel: Country selector, metric selector, and date range
    html.Div([
        # Drop down to select country 
        html.Label("Select a Country:"),
        dcc.Dropdown(
            options=[{'label': c, 'value': c} for c in sorted(df['country'].unique())],
            value='United States',
            id='country-dropdown'
        ),

        # Radio buttons to select vaccination metric
        html.Label("Select Metric:"),
        dcc.RadioItems(
            options=[
                {'label': 'People Fully Vaccinated', 'value': 'people_fully_vaccinated'},
                {'label': 'Daily Vaccinations', 'value': 'daily_vaccinations'}
            ],
            value='people_fully_vaccinated',
            id='metric-radio'
        ),

        # Date range picker to filter data
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=df['date'].min(),
            max_date_allowed=df['date'].max(),
            start_date=df['date'].min(),
            end_date=df['date'].max()
        ),
    ], style={'width': '50%', 'margin': 'auto'}),

    # Line chart revealing vaccination trends over time
    dcc.Graph(id='line-graph'),
    
    # bar chart showing the top 10 countries by fully vaccinated population
    html.H2("Top 10 Countries by Vaccination", style={'textAlign': 'center'}),
    dcc.Graph(
        id='bar-graph',
        figure=px.bar(
            latest_df.nlargest(10, 'people_fully_vaccinated'),
            x='country',
            y='people_fully_vaccinated',
            labels={'people_fully_vaccinated': 'Fully Vaccinated People'},
            title='Top 10 Countries (Latest Date)',
            template='plotly_white'
        )
    ),

    # Choropleth map showing global distribution of vaccinations
    html.H2("Global Vaccination Map", style={'textAlign': 'center'}),
    dcc.Graph(
        id='map-graph',
        figure=px.choropleth(
            latest_df,
            locations='iso_code',
            color='people_fully_vaccinated',
            hover_name='country',
            color_continuous_scale='Viridis',
            title='Global Vaccinations (Latest Date)',
            template='plotly_white'
        )
    )
])


In [15]:
# Callback
'''
This call back updates the line chart based on user input: 
- Country from dropdown
- Vaccination metric from radio buttons
- Date range from date picker
'''
@app.callback(
    Output('line-graph', 'figure'),
    Input('country-dropdown', 'value'),
    Input('metric-radio', 'value'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date')
)
def update_line_chart(selected_country, selected_metric, start_date, end_date):
    '''
    Generates an updated line chart showing the selected vaccination metric 
    for the chosen country and date range. 

    Parameters: 
    - Selected_country (str): Country selected from dropdown
    - selected_metric (str): Vaccination metric selected
    - start_date (str): Start date of selected range
    - end_date (str): End date of selected range

    Returns: 
    - fig (pltly.graph_objects.Figure): Line chart with filtered data 

    '''

    # Filter the DataFrame based on selected country and date range
    filtered = df[
        (df['country'] == selected_country) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]

    # Creating line chart using plotly express
    fig = px.line(
        filtered,
        x='date',
        y=selected_metric,
        title=f"{selected_metric.replace('_', ' ').title()} in {selected_country}",
        labels={selected_metric: selected_metric.replace('_', ' ').title()},
        template='plotly_white'
    )
    return fig


In [16]:
if __name__ == '__main__':
    app.run(debug=True)